# FastKafka

This notebook will demonstrate the capabilities and developed functionalities in FastKafka project


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/airtai/fastkafka/blob/64-colab-based-tutorial/nbs/guides/Guide_00_FastKafka_Demo.ipynb)

## Installing fastkafka library

To install fastkafka, run: `pip install fastkafka` in your terminal

In [ ]:
try:
    import fastkafka
except ImportError:
    #!pip install fastkafka==0.1.0
    !pip install "fastkafka @ git+https://github.com/airtai/fastkafka@c082b8264819c11183cae89b2868f6bc51350371"

## LocalKafkaBroker

To be able to test and demonstrate the use of FastKafka, we have developed a python wrapper for Zookeeper and Kafka broker which is demonstrated here and used later in the notebook. 

In [ ]:
from fastkafka.testing import LocalKafkaBroker

First, start the LocalKafkaBroker

When LocalKafkaBroker is started, it checks if there are Java and Kafka installed on the system, if not, it will install them and export them to path as it is necessary for it to function.

Note: We use `apply_nest_asyncio=True` when creating the broker in the notebook to enable it to run in a nested async loop

In [ ]:
local_broker = LocalKafkaBroker(apply_nest_asyncio=True)
bootstrap_server = local_broker.start()
print(bootstrap_server)

[INFO] fastkafka.testing: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting kafka...
[INFO] fastkafka.testing: kafka started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.testing: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): exited.
12

Lets see if there are any topics in our fresh Kafka broker. If everything is okay, there should be none.

In [ ]:
! kafka-topics.sh --list --bootstrap-server {bootstrap_server}

Lets now create a topic, list it, and describe it to see that our LocalKafkaBroker is really running.

In [ ]:
! kafka-topics.sh --create --topic quickstart-events --bootstrap-server {bootstrap_server}

Created topic quickstart-events.


In [ ]:
! kafka-topics.sh --list --bootstrap-server {bootstrap_server}

quickstart-events


In [ ]:
! kafka-topics.sh --describe --topic quickstart-events --bootstrap-server {bootstrap_server}

Topic: quickstart-events	TopicId: 3vDYPDnKS36PEHf2o6im6A	PartitionCount: 1	ReplicationFactor: 1	Configs: flush.ms=1000,segment.bytes=1073741824,flush.messages=10000,retention.bytes=1073741824
	Topic: quickstart-events	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


Now we can stop the broker as it is no longer needed

In [ ]:
local_broker.stop()

[INFO] fastkafka.testing: LocalKafkaBroker.stop(): entering...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 41096...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 41096 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 40713...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 40713 terminated.
[INFO] fastkafka.testing: LocalKafkaBroker.stop(): exited.


LocalKafkaBroker can also be used as a context manager

In [ ]:
with LocalKafkaBroker(apply_nest_asyncio=True) as bootstrap_server:
    print(bootstrap_server)

[INFO] fastkafka.testing: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting kafka...
[INFO] fastkafka.testing: kafka started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.testing: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): exited.
127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.stop(): entering...
[INFO] fastkafka._components._subprocess: terminate_as

## FastKafka demo

Now we will create a fastkafka application containing a Model that will ingest data samples from one Kafka topic (input_data) and produce predictions to another Kafka topic (predictions)

### Preparing the demo model

First we will prepare our model with the Iris dataset so that we can demonstrate the preditions using FastKafka

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
model = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
x = X[[0, 55, -1]]
print(x)
print(model.predict(x))

[[5.1 3.5 1.4 0.2]
 [5.7 2.8 4.5 1.3]
 [5.9 3.  5.1 1.8]]
[0 1 2]


Now, we need to model the input and prediction messages that will be sent to the Kafka broker

In [ ]:
from pydantic import BaseModel, NonNegativeFloat, Field

In [ ]:
class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPredictionData(BaseModel):
    species: str = Field(..., example="Iris-setosa", description="Predicted species")

Now, lets prepare our prediction FastKafka app.

In [ ]:
from fastkafka.application import FastKafka

In [ ]:
kafka_app = FastKafka()

iris_species = ["setosa", "versicolor", "Iris-virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest", group_id="my_group")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPredictionData:
    prediction = IrisPredictionData(species=iris_species[species_class])
    return prediction

Lets run the test by sending a message to the running app that now encapsulates the Iris classification model:

In [ ]:
from fastkafka.application import Tester

In [ ]:
msg = IrisInputData(
    sepal_length=0.1,
    sepal_width=0.2,
    petal_length=0.3,
    petal_width=0.4,
)

with LocalKafkaBroker(
    topics=["input_data", "predictions"], apply_nest_asyncio=True
) as bootstrap_servers:
    kafka_app.set_bootstrap_servers(bootstrap_servers=bootstrap_servers)
    async with Tester(kafka_app) as tester:
        await tester.to_input_data(msg)
        await tester.awaited_mocks.on_predictions.assert_awaited(timeout=2)
        prediction = tester.mocks.on_predictions.call_args

print("*"*100)
print(f"Sent data: {msg}")
print(f"Received prediction: {prediction}")

[INFO] fastkafka.testing: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting kafka...
[INFO] fastkafka.testing: kafka started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.testing: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): exited.
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
[INF

### Recap

We have created a Iris classification model and encapulated it into our fastkafka application.
The app will consume the IrisInputData from the `input_data` topic and produce the predictions to `predictions` topic.

To test the app we have:
1. Created the app
1. Started the LocalKafkaBroker
2. Started our Tester class which mirrors the developed app topics for testing purpuoses
3. Sent IrisInputData message to `input_data` topic
4. Asserted and checked that the developed iris classification service has reacted to IrisInputData message 

## Documentation

The kafka app comes with builtin documentation generation, let's demonstrate that.

To generate the documentation programatically you just need to call `.generate_docs()` method of your app. This will generate the *asyncapi* folder in relative path where all your documentation will be saved. 

In [ ]:
kafka_app.create_docs()

[INFO] fastkafka._components.asyncapi: Old async specifications at '/work/fastkafka/nbs/guides/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/work/fastkafka/nbs/guides/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /work/fastkafka/nbs/guides/asyncapi/docs.




In [ ]:
! ls asyncapi/

docs  spec


In docs folder you will fint the serveable static html file of your documentation. This can also be served using our `fastkafka docs serve` CLI command (more on that in our guides).

In spec folder you will find a asyncapi.yml file containing the async API specification of your application. We will now generate a link to AsyncAPI from the docs that will load your documentation for quick inspection.

In [ ]:
import base64

In [ ]:
with open("asyncapi/spec/asyncapi.yml", "r") as async_specs:
    specs = async_specs.read()
    base64_specs = base64.b64encode(specs.encode("UTF-8")).decode("UTF-8")
    print(f"https://studio.asyncapi.com/?base64={base64_specs}")

https://studio.asyncapi.com/?base64=YXN5bmNhcGk6IDIuNS4wCmluZm86CiAgdGl0bGU6ICcnCiAgdmVyc2lvbjogJycKICBkZXNjcmlwdGlvbjogJycKICBjb250YWN0OgogICAgbmFtZTogQXV0aG9yCiAgICB1cmw6IGh0dHBzOi8vd3d3Lmdvb2dsZS5jb20KICAgIGVtYWlsOiBub3JlcGx5QGdtYWlsLmNvbQpzZXJ2ZXJzOgogIGxvY2FsaG9zdDoKICAgIHVybDogaHR0cHM6Ly9sb2NhbGhvc3QKICAgIGRlc2NyaXB0aW9uOiBMb2NhbCAoZGV2KSBLYWZrYSBicm9rZXIKICAgIHByb3RvY29sOiBrYWZrYQogICAgdmFyaWFibGVzOgogICAgICBwb3J0OgogICAgICAgIGRlZmF1bHQ6ICc5MDkyJwpjaGFubmVsczoKICBpbnB1dF9kYXRhOgogICAgc3Vic2NyaWJlOgogICAgICBtZXNzYWdlOgogICAgICAgICRyZWY6ICcjL2NvbXBvbmVudHMvbWVzc2FnZXMvSXJpc0lucHV0RGF0YScKICBwcmVkaWN0aW9uczoKICAgIHB1Ymxpc2g6CiAgICAgIG1lc3NhZ2U6CiAgICAgICAgJHJlZjogJyMvY29tcG9uZW50cy9tZXNzYWdlcy9JcmlzUHJlZGljdGlvbkRhdGEnCmNvbXBvbmVudHM6CiAgbWVzc2FnZXM6CiAgICBJcmlzSW5wdXREYXRhOgogICAgICBwYXlsb2FkOgogICAgICAgIHRpdGxlOiBJcmlzSW5wdXREYXRhCiAgICAgICAgdHlwZTogb2JqZWN0CiAgICAgICAgcHJvcGVydGllczoKICAgICAgICAgIHNlcGFsX2xlbmd0aDoKICAgICAgICAgICAgdGl0bGU6IFNlcGFsIExlbmd0aAogICAgICAgICAgICBkZXNj

1. Create app

In [ ]:
app = create_app()

2. Start the broker

In [ ]:
broker = LocalKafkaBroker(topics=["input_data", "predictions"], apply_nest_asyncio=True)
bootstrap_server = broker.start()
app.set_bootstrap_servers(bootstrap_server)

3. Started our Tester class which mirrors the developed app topics for testing purpuoses

In [ ]:
tester = Tester(app)
await tester.__aenter__()

4. Send a message and see what we get at the predictions topic

In [ ]:
msg = IrisInputData(
    sepal_length=X[0][0],
    sepal_width=X[0][1],
    petal_length=X[0][2],
    petal_width=X[0][3],
)

await tester.to_input_data(msg)
await tester.awaited_mocks.on_predictions.assert_awaited(timeout=2)
print(f"Sent data: {msg}")
print(f"Received prediction: {tester.mocks.on_predictions.call_args}")

5. To keep everything clean, close the broker and tester

In [ ]:
await tester.__aexit__(None, None, None)
broker.stop()

When condensed into one cell, the test looks like this: